In [7]:
import re
from itertools import chain

import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

from textmining.lemmatization import Lemmas
from textmining.tokenization import tokenize

In [2]:
lemmas = Lemmas.from_file()

In [3]:
tokenizer = AutoTokenizer.from_pretrained("henryk/bert-base-multilingual-cased-finetuned-polish-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("henryk/bert-base-multilingual-cased-finetuned-polish-squad2")

Downloading: 100%|██████████| 679M/679M [00:23<00:00, 30.7MB/s] 


In [20]:
tokenizer.decode(inputs.input_ids[0])

'[CLS] Dokończ przysłowie : „ pierwsze koty za... [UNK] [SEP] pierwsze koty za płoty [SEP]'

In [83]:
question, text = "Za panowania którego polskiego króla według przysłowia wyginęła szlachta?", "za króla Olbrachta wyginęła szlachta"
inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'Olbrachta'

In [33]:
def is_proverb_question(question: str):
    if set(chain.from_iterable(map(lemmas.lemmatize, tokenize(question.lower())))).intersection(lemmas.lemmatize("przysłowie")):
        return True
    return False

In [48]:
with open("data/proverbs.txt") as f:
    PROVERBS = f.read()

In [ ]:
def normalize(text: str):
    return " ".join(tokenize(text))

In [217]:
from sklearn.feature_extraction.text import TfidfVectorizer

def _tfidf_tokenize(text: str):
    return list(chain.from_iterable([lemmas.lemmatize(token) for token in tokenize(text.lower())]))

TFIDF_VECTORIZER = TfidfVectorizer(tokenizer=_tfidf_tokenize)
PROVERBS_TFIDF = TFIDF_VECTORIZER.fit_transform(PROVERBS.strip().splitlines())

def get_most_similar_proverbs(question: str, k: int = 10):
    question_tfidf = TFIDF_VECTORIZER.transform([question])
    terms_idxs = question_tfidf.nonzero()[1]
    proverbs_tfidf_sum = PROVERBS_TFIDF[:, terms_idxs].sum(1).A1
    best_proverbs_idxs = proverbs_tfidf_sum.argsort()[-1:-k:-1]
    return " [SEP] ".join(PROVERBS.strip().splitlines()[idx] for idx in best_proverbs_idxs)

In [235]:
def handle_proverb_question(question):
    question_lemmas = set(chain.from_iterable(map(lemmas.lemmatize, tokenize(question.lower()))))

    if question_lemmas.intersection(lemmas.lemmatize("dokończ")):
        match = re.search("„(?P<name>.*)...”", question.lower())
        if match:
            beginning = match["name"]
            beginning_match = re.search(f"^{beginning}(?P<name>.*)$", PROVERBS.lower(), flags=re.MULTILINE)
            if beginning_match:
                return normalize(beginning_match["name"])

    most_similar = get_most_similar_proverbs(question.lower())
    inputs = tokenizer(question.lower(), most_similar, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    if answer_start_index != 0:
        offset = outputs.end_logits[0, answer_start_index:].argmax()

        predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_start_index + offset + 1]
        return(tokenizer.decode(predict_answer_tokens))
    return None

In [236]:
print(handle_proverb_question("Proszę dokończyć przysłowie: „nosił wilk razy kilka...”"))
print(handle_proverb_question("Za panowania którego polskiego króla według przysłowia wyginęła szlachta?"))

ponieśli i wilka
Olbrachta


In [237]:
with open("data/poleval/pytania.txt", "rt") as f_q, open("data/poleval/odpowiedzi.txt", "rt") as f_a:
    for q, a in zip(f_q.readlines(), f_a.readlines()):
        if set(chain.from_iterable(map(lemmas.lemmatize, tokenize(q.lower())))).intersection(lemmas.lemmatize("przysłowie")):
            print("Q:", q.strip(), "A:", handle_proverb_question(q), "(Correct:", a.strip(), ")")

Q: Według przysłowia nos jest dla tabakiery czy tabakiera dla nosa? A: None (Correct: tabakiera dla nosa )
Q: Dokończ przysłowie: „dobry zwyczaj: nie...”? A: None (Correct: pożyczaj )
Q: Co według przysłowia służy za przewodnika? A: None (Correct: koniec języka )
Q: Dokończ przysłowie: „pierwsze koty za...” A: płoty (Correct: płoty )
Q: Czego według przysłowia pragnie kania? A: None (Correct: dżdżu )
Q: Czego według przysłowia wart jest pac? A: None (Correct: pałaca )
Q: Które zwierzę według przysłowia chodzi swoimi własnymi ścieżkami? A: None (Correct: kot )
Q: Proszę dokończyć przysłowie: „nosił wilk razy kilka...” A: ponieśli i wilka (Correct: ponieśli i wilka )
Q: Za panowania którego polskiego króla według przysłowia wyginęła szlachta? A: Olbrachta (Correct: Jana Olbrachta )
